In [1]:
import numpy as np
import pandas as pd

In [5]:
# Create a dictionary of DH parameters
dh_parameters = {
    'Link': ['Link 1', 'Link 2', 'Link 3', 'Link 4', 'Link 5', 'Link 6'],
    'theta': [0, 90, 0, 0, -90, 0],
    'd': [0, 0.301, 0, 0, 0.50, 0.170],
    'a': [0.275, 0, 0.700, .190, 0, 0],
    'alpha': [0, 90, 0, 90, -90, 90]
}

# Create the DataFrame
df = pd.DataFrame(dh_parameters)

# Set the 'Link' column as the index
df.set_index('Link', inplace=True)

# Display the DataFrame
print(df)


        theta      d      a  alpha
Link                              
Link 1      0  0.000  0.275      0
Link 2     90  0.301  0.000     90
Link 3      0  0.000  0.700      0
Link 4      0  0.000  0.190     90
Link 5    -90  0.500  0.000    -90
Link 6      0  0.170  0.000     90


In [8]:
def calculate_dh_parameters(link_lengths, link_offsets, joint_angles, link_twists):
    num_links = len(link_lengths)
    dh_parameters = []

    for i in range(num_links):
        a = link_lengths[i]
        d = link_offsets[i]
        theta = joint_angles[i]
        alpha = link_twists[i]

        dh_matrix = np.array([
            [np.cos(theta), -np.sin(theta) * np.cos(alpha), np.sin(theta) * np.sin(alpha), a * np.cos(theta)],
            [np.sin(theta), np.cos(theta) * np.cos(alpha), -np.cos(theta) * np.sin(alpha), a * np.sin(theta)],
            [0, np.sin(alpha), np.cos(alpha), d],
            [0, 0, 0, 1]
        ])

        dh_parameters.append({
            'a': a,
            'alpha': alpha,
            'd': d,
            'theta': theta,
            'dh_matrix': dh_matrix
        })

    return dh_parameters

# Example usage
link_lengths = [0.275, 0, 0.7, 0.19, 0, 0]
link_offsets = [0, 0.301, 0, 0, 0.5, 0.17]
joint_angles = [0.1, 0.2 + np.pi/2 , 0.3, 0.4, 0.5 - np.pi/2, 0.6]
link_twists = [0, np.pi/2, 0, np.pi/2, -np.pi/2, 0]

dh_parameters = calculate_dh_parameters(link_lengths, link_offsets, joint_angles, link_twists)

# Print the DH parameters
for i, params in enumerate(dh_parameters):
    print(f"Link {i+1}:")
    print("a =", params['a'])
    print("alpha =", params['alpha'])
    print("d =", params['d'])
    print("theta =", params['theta'])
    print("DH matrix:")
    print(params['dh_matrix'])
    print()


Link 1:
a = 0.275
alpha = 0
d = 0
theta = 0.1
DH matrix:
[[ 0.99500417 -0.09983342  0.          0.27362615]
 [ 0.09983342  0.99500417 -0.          0.02745419]
 [ 0.          0.          1.          0.        ]
 [ 0.          0.          0.          1.        ]]

Link 2:
a = 0
alpha = 1.5707963267948966
d = 0.301
theta = 1.7707963267948965
DH matrix:
[[-1.98669331e-01 -6.00117699e-17  9.80066578e-01 -0.00000000e+00]
 [ 9.80066578e-01 -1.21649880e-17  1.98669331e-01  0.00000000e+00]
 [ 0.00000000e+00  1.00000000e+00  6.12323400e-17  3.01000000e-01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]

Link 3:
a = 0.7
alpha = 0
d = 0
theta = 0.3
DH matrix:
[[ 0.95533649 -0.29552021  0.          0.66873554]
 [ 0.29552021  0.95533649 -0.          0.20686414]
 [ 0.          0.          1.          0.        ]
 [ 0.          0.          0.          1.        ]]

Link 4:
a = 0.19
alpha = 1.5707963267948966
d = 0
theta = 0.4
DH matrix:
[[ 9.21060994e-01 -2.38449963e-17  3.8941834

In [17]:

def calculate_homogeneous_matrix(a, alpha, d, theta):
    cos_theta = np.cos(theta)
    sin_theta = np.sin(theta)
    cos_alpha = np.cos(alpha)
    sin_alpha = np.sin(alpha)

    # Construct the homogeneous transformation matrix
    homogeneous_matrix = np.array([
        [cos_theta, -sin_theta * cos_alpha, sin_theta * sin_alpha, a * cos_theta],
        [sin_theta, cos_theta * cos_alpha, -cos_theta * sin_alpha, a * sin_theta],
        [0, sin_alpha, cos_alpha, d],
        [0, 0, 0, 1]
    ])

    return homogeneous_matrix

def calculate_homogeneous_matrices(dh_parameters):
    num_links = len(dh_parameters)
    homogeneous_matrices = []

    # Calculate the homogeneous transformation matrix for each link
    for i in range(num_links):
        a = dh_parameters[i]['a']
        alpha = dh_parameters[i]['alpha']
        d = dh_parameters[i]['d']
        theta = dh_parameters[i]['theta']

        homogeneous_matrix = calculate_homogeneous_matrix(a, alpha, d, theta)
        homogeneous_matrices.append(homogeneous_matrix)

    return homogeneous_matrices


def calculate_robot_homogeneous_matrix(homogeneous_matrices):
    # Multiply the individual homogeneous matrices to get the overall homogeneous matrix
    robot_homogeneous_matrix = np.eye(4)

    for matrix in homogeneous_matrices:
        robot_homogeneous_matrix = np.dot(robot_homogeneous_matrix, matrix)

    return robot_homogeneous_matrix



# DH parameters for Motoman HC10DPT Robot
dh_parameters = [
    {'a': 0.275, 'alpha': 0, 'd': 0, 'theta': 0.1},
    {'a': 0, 'alpha': np.pi/2, 'd': 0.301, 'theta': 0.2 + np.pi/2 },
    {'a': 0.7, 'alpha': 0, 'd': 0, 'theta': 0.3},
    {'a': 0.19, 'alpha': np.pi/2, 'd': 0, 'theta': 0.4},
    {'a': 0.0, 'alpha': -np.pi/2, 'd': 0.5, 'theta': 0.5 - np.pi/2},
    {'a': 0, 'alpha': np.pi/2, 'd': 0.17, 'theta': 0.6}
]


homogeneous_matrices = calculate_homogeneous_matrices(dh_parameters)
robot_homogeneous_matrix = calculate_robot_homogeneous_matrix(homogeneous_matrices)


# Print the DH parameters for the robot
robot = pd.DataFrame(dh_parameters)
print(robot, end='\n')


# Print the homogeneous matrices for each link
for i, matrix in enumerate(homogeneous_matrices):
    print(f"Homogeneous matrix for Link {i+1}:")
    print(matrix)
    print()


# Print the overall homogeneous matrix for the robot
print("Overall Homogeneous Matrix for the Robot:")
print(robot_homogeneous_matrix)

       a     alpha      d     theta
0  0.275  0.000000  0.000  0.100000
1  0.000  1.570796  0.301  1.770796
2  0.700  0.000000  0.000  0.300000
3  0.190  1.570796  0.000  0.400000
4  0.000 -1.570796  0.500 -1.070796
5  0.000  1.570796  0.170  0.600000
Homogeneous matrix for Link 1:
[[ 0.99500417 -0.09983342  0.          0.27362615]
 [ 0.09983342  0.99500417 -0.          0.02745419]
 [ 0.          0.          1.          0.        ]
 [ 0.          0.          0.          1.        ]]

Homogeneous matrix for Link 2:
[[-1.98669331e-01 -6.00117699e-17  9.80066578e-01 -0.00000000e+00]
 [ 9.80066578e-01 -1.21649880e-17  1.98669331e-01  0.00000000e+00]
 [ 0.00000000e+00  1.00000000e+00  6.12323400e-17  3.01000000e-01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]

Homogeneous matrix for Link 3:
[[ 0.95533649 -0.29552021  0.          0.66873554]
 [ 0.29552021  0.95533649 -0.          0.20686414]
 [ 0.          0.          1.          0.        ]
 [ 0.          0.         